In [2]:
import os
from openai import AzureOpenAI
from openai import OpenAI
import time
from dataclasses import dataclass
import pandas as pd
import json
import dill


In [3]:
#我的gpt4通过“Azure”获取的，所以用到AzureOpenAI
#model：gpt-4
client_gpt = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)
#我的kimi和doubao都是在“火山方舟”获取的
#对于Moonshot-v1-8k-v1，model：ep-20250113135755-7rfkr
#对于DouBao-pro-4k-240515，model：ep-20250113134522-vfmmb
client_kimi_doubao = OpenAI(
    base_url="https://ark.cn-beijing.volces.com/api/v3",
    api_key=os.getenv("ARK_API_KEY")#方舟api_key

)
simple_data=pd.read_csv('../datasets/simple_choice_questions.csv')
multiple_data=pd.read_csv('../datasets/multiple_choice_questions.csv')

In [26]:
@dataclass
class QuestionResult:
    index: int              # 问题的索引,从0开始
    question: str           # 问题内容
    correct_answer: str     # 正确答案
    predicted_answer: str=""   # 模型预测的答案
    error_info: str=""              # 错误内容（如果有错误的话）

def print_result(questionResult ):
        # 打印输入和模型的回答
        print(f"索引: {questionResult.index}")
        print(f"问题: {questionResult.question}")
        print(f"正确答案: {questionResult.correct_answer}")
        if questionResult.predicted_answer!="":
            print(f"模型的回答: {questionResult.predicted_answer}")
        if questionResult.error_info!="":
            print(f"错误信息: {questionResult.error_info}")

def simple_choice(df,client,model):
    correct_list =[]
    incorrect_list = []
    error_list = []
    # 遍历每一行数据进行模型预测和答案比较
    for index, row in df.iterrows():
        input_text = f"""
        {row['input']}
        A: {row['A']}
        B: {row['B']}
        C: {row['C']}
        D: {row['D']}
        """
        correct_answer = row['output']
        result=QuestionResult(
            index=index,
            question=input_text,
            correct_answer=correct_answer,
        )
        # 构造对话内容
        messages = [
            {"role": "system", "content": "请完成如下单项选择题，你的回答应只有一个选项字母且不包含任何标点符号文字（如：A）"},
            {"role": "user", "content": input_text}
        ]
        error_info=''
        while  True:
            try:
                response = client.chat.completions.create(
                    model=model,
                    messages=messages,
                )
            except Exception as e:
                error_info=e
            if error_info!="Connection error." and error_info!="Request timed out.":
                break
        if error_info!="Connection error." and error_info!="" and error_info!="Request timed out.":
            result.error_info=error_info
            print_result(result)
            error_list.append(result)
            continue
        # 获取模型的预测结果
        predicted_answer = response.choices[0].message.content.strip()
        # 检查模型的回答是否正确
        if predicted_answer.strip().startswith(correct_answer):
            result.predicted_answer=predicted_answer
            print_result(result)
            correct_list.append(result)
        else:
            result.predicted_answer=predicted_answer
            print_result(result)
            incorrect_list.append(result)
        time.sleep(1)
    return correct_list,incorrect_list,error_list
def multiple_choice(df,client,model):
    correct_list =[]
    incorrect_list = []
    error_list = []
    # 遍历每一行数据进行模型预测和答案比较
    for index, row in df.iterrows():
        input_text = f"""
        {row['input']}
        A: {row['A']}
        B: {row['B']}
        C: {row['C']}
        D: {row['D']}
        """
        correct_answer = row['output']
        result=QuestionResult(
            index=index,
            question=input_text,
            correct_answer=correct_answer,
        )
        # 构造对话内容
        messages = [
            {"role": "system", "content": "请完成如下多项选择题，答案将包括2到4个选项，你的回答应直接由选项字母组成且不包含任何标点符号文字（如：ABCD）"},
            {"role": "user", "content": input_text}
        ]
        error_info=''
        while  True:
            try:
                response = client.chat.completions.create(
                    model=model,
                    messages=messages,
                )
            except Exception as e:
                error_info=e
            if error_info!="Connection error." and error_info!="Request timed out.":
                break
        if error_info!="Connection error." and error_info!="" and error_info!="Request timed out.":
            result.error_info=error_info
            print_result(result)
            error_list.append(result)
            continue
            
        # 获取模型的预测结果
        predicted_answer = response.choices[0].message.content.strip()    
        # 检查模型的回答是否正确
        if predicted_answer.strip().startswith(correct_answer):
            result.predicted_answer=predicted_answer
            print_result(result)
            correct_list.append(result)
        else:
            result.predicted_answer=predicted_answer
            print_result(result)
            incorrect_list.append(result)
        time.sleep(1)
    return correct_list,incorrect_list,error_list


In [17]:
doubao_simple_correct_list,doubao_simple_incorrect_list,doubao_simple_error_list=simple_choice(simple_data,client_kimi_doubao,"ep-20250113134522-vfmmb")
doubao_multiple_correct_list,doubao_multiple_incorrect_list,doubao_multiple_error_list=multiple_choice(multiple_data,client_kimi_doubao,"ep-20250113134522-vfmmb")
print(len(doubao_simple_correct_list),len(doubao_simple_incorrect_list),len(doubao_simple_error_list))
print(len(doubao_multiple_correct_list),len(doubao_multiple_incorrect_list),len(doubao_multiple_error_list))
# 将列表保存到文件
lists_to_save = {
    "doubao_simple_correct_list": doubao_simple_correct_list,
    "doubao_simple_incorrect_list": doubao_simple_incorrect_list,
    "doubao_simple_error_list": doubao_simple_error_list,
    "doubao_multiple_correct_list": doubao_multiple_correct_list,
    "doubao_multiple_incorrect_list": doubao_multiple_incorrect_list,
    "doubao_multiple_error_list": doubao_multiple_error_list,
}
with open("../result/doubao_lists.json", "w") as f:
    json.dump(lists_to_save, f)

问题: 
        2019 年 4 月 10 日晚 9 点，全球多国科研人员合作的“事件视界望远镜”项目在全球六地同步举行新闻发布会，发布了世界上首张黑洞图像，公布了人类首次拍到的黑洞照片，这 是继 2015 年人类通过引力波视测“听到”两个“黑洞”合体之后，证明黑洞存在的直接“视界” 证据。有科学家认为，这张看起来有点模糊的照片意义非凡，它再次验证了爱因斯坦广义相 对论的预言是对的，并将进一步帮助科学家解答星系演化等一系列宇宙本质问题。人类首次 “看到”黑洞正面照表明( )
        A: 空间的性质依赖于物质的分布及其运动状态  
        B: 世界是物质的统一体
        C: 物质世界的客观存在与人的实践和认识水平有关  
        D: 空间的观念随着人的认识发展而拓展
        
正确答案: AB
模型的回答: ABCD
索引: 0
问题: 
        2019 年 4 月 10 日晚 9 点，全球多国科研人员合作的“事件视界望远镜”项目在全球六地同步举行新闻发布会，发布了世界上首张黑洞图像，公布了人类首次拍到的黑洞照片，这 是继 2015 年人类通过引力波视测“听到”两个“黑洞”合体之后，证明黑洞存在的直接“视界” 证据。有科学家认为，这张看起来有点模糊的照片意义非凡，它再次验证了爱因斯坦广义相 对论的预言是对的，并将进一步帮助科学家解答星系演化等一系列宇宙本质问题。人类首次 “看到”黑洞正面照表明( )
        A: 空间的性质依赖于物质的分布及其运动状态  
        B: 世界是物质的统一体
        C: 物质世界的客观存在与人的实践和认识水平有关  
        D: 空间的观念随着人的认识发展而拓展
        
正确答案: AB
模型的回答: ABCD
问题: 
        2019 年 10 月 24 日，中共中央政治局就区块链技术发展现状和趋势进行集体学习。习近平在主持学习时指出， 我们要提高运用和管理区块链技术能力，使区块链技术在建设网络强国、发展数字经济、助力经济社会发展等方面发挥更大作用”。有专家认为，基于区块链技术，人们可以构建在没有中介辅助下多个参与方之间的资产交易、价值传递的网络，通过建立执行智能合约，推动契约关系和规则的维护和履行，降低信用建立

In [19]:
kimi_simple_correct_list,kimi_simple_incorrect_list,kimi_simple_error_list=simple_choice(simple_data,client_kimi_doubao,"ep-20250113135755-7rfkr")
kimi_multiple_correct_list,kimi_multiple_incorrect_list,kimi_multiple_error_list=multiple_choice(multiple_data,client_kimi_doubao,"ep-20250113135755-7rfkr")
print(len(kimi_simple_correct_list),len(kimi_simple_incorrect_list),len(kimi_simple_error_list))
print(len(kimi_multiple_correct_list),len(kimi_multiple_incorrect_list),len(kimi_multiple_error_list))
# 将列表保存到文件
kimi_lists_to_save = {
    "kimi_simple_correct_list": kimi_simple_correct_list,
    "kimi_simple_incorrect_list": kimi_simple_incorrect_list,
    "kimi_simple_error_list": kimi_simple_error_list,
    "kimi_multiple_correct_list": kimi_multiple_correct_list,
    "kimi_multiple_incorrect_list": kimi_multiple_incorrect_list,
    "kimi_multiple_error_list": kimi_multiple_error_list,
}
with open("../result/kimi_lists.json", "w") as f:
    json.dump(kimi_lists_to_save, f)

索引: 0
问题: 
        恩格斯于 1820 年 11 月 28 日出生在德国巴门市的一个工厂主家庭。他称自己一生所做的事就是“拉第二小提琴"。  恩格斯不仅与马克思一起创立马克思主 义，参加并指导国际工人运动，而且在传播和发展马克思主义方面作出了杰出的贡献。恩格斯全面阐述马克思主义理论体系的著作是(  )
        A: 《共产党宣言》
        B: 家庭、私有制和国家的起源
        C: 《反杜林论》
        D: 《自然辩证法》
        
正确答案: C
模型的回答: C
索引: 1
问题: 
        逐渐“飞入寻常百姓家”的人工智能对人类的社会生活产生了巨大影响，对此，技术乐观派 认为人工智能为人类发展更新了“发动机”，悲观派则认为人工智能的发展可能导致“人类的终结“。对人工智能社会效用的评价是( )
        A: 对客观事物发展规律的真理性评价 
        B: 对主体需要发展程度的认知性评价
        C:  受主体情感、兴趣和爱好影响的主观性评价
        D:  对客观事物的属性及其满足主体需要的价值性评价
        
正确答案: D
模型的回答: D
索引: 2
问题: 
        在相同的劳动时间里，复杂劳动创造的价值大于简单劳动创造的价值。在以私有制为基础 的商品经济条件下，复杂劳动转化为简单劳动( )
        A: 是商品生产者自觉计算出来的  
        B: 是市场管理部门事先规定的 
        C: 是商品生产者协商出来的
        D: 是商品交换过程中市场机制自发实现的
        
正确答案: D
模型的回答: D
索引: 3
问题: 
        十月革命之前，列宁总结当时变化了的新情况，得出“社会主义可能首先在少数甚至在单独一个资本主文国家内获得胜利”的结论。列宁得出这个结论的理论依据是( )
        A: 资本主义经济和政治发展不平衡规律 
        B: 资本主义国家的不断衰落
        C: 科学社会主义理论的广泛传播 
        D: 无产阶级力量的不断壮大
        
正确答案: A
模型的回答: A
索引: 4
问题: 
        “什么是社会主义?

In [6]:
gpt4_simple_correct_list,gpt4_simple_incorrect_list,gpt4_simple_error_list=simple_choice(simple_data,client_gpt,"gpt-4")
gpt4_multiple_correct_list,gpt4_multiple_incorrect_list,gpt4_multiple_error_list=multiple_choice(multiple_data,client_gpt,"gpt-4")
print(len(gpt4_simple_correct_list),len(gpt4_simple_incorrect_list),len(gpt4_simple_error_list))
print(len(gpt4_multiple_correct_list),len(gpt4_multiple_incorrect_list),len(gpt4_multiple_error_list))
gpt4_lists_to_save = {
    "gpt4_simple_correct_list": gpt4_simple_correct_list,
    "gpt4_simple_incorrect_list": gpt4_simple_incorrect_list,
    "gpt4_simple_error_list": gpt4_simple_error_list,
    "gpt4_multiple_correct_list": gpt4_multiple_correct_list,
    "gpt4_multiple_incorrect_list": gpt4_multiple_incorrect_list,
    "gpt4_multiple_error_list": gpt4_multiple_error_list,
}
with open('../result/gpt4_lists.pkl', 'wb') as f:
    dill.dump(gpt4_lists_to_save, f)


索引: 0
问题: 
        恩格斯于 1820 年 11 月 28 日出生在德国巴门市的一个工厂主家庭。他称自己一生所做的事就是“拉第二小提琴"。  恩格斯不仅与马克思一起创立马克思主 义，参加并指导国际工人运动，而且在传播和发展马克思主义方面作出了杰出的贡献。恩格斯全面阐述马克思主义理论体系的著作是(  )
        A: 《共产党宣言》
        B: 家庭、私有制和国家的起源
        C: 《反杜林论》
        D: 《自然辩证法》
        
正确答案: C
模型的回答: C
索引: 1
问题: 
        逐渐“飞入寻常百姓家”的人工智能对人类的社会生活产生了巨大影响，对此，技术乐观派 认为人工智能为人类发展更新了“发动机”，悲观派则认为人工智能的发展可能导致“人类的终结“。对人工智能社会效用的评价是( )
        A: 对客观事物发展规律的真理性评价 
        B: 对主体需要发展程度的认知性评价
        C:  受主体情感、兴趣和爱好影响的主观性评价
        D:  对客观事物的属性及其满足主体需要的价值性评价
        
正确答案: D
模型的回答: D
索引: 2
问题: 
        在相同的劳动时间里，复杂劳动创造的价值大于简单劳动创造的价值。在以私有制为基础 的商品经济条件下，复杂劳动转化为简单劳动( )
        A: 是商品生产者自觉计算出来的  
        B: 是市场管理部门事先规定的 
        C: 是商品生产者协商出来的
        D: 是商品交换过程中市场机制自发实现的
        
正确答案: D
模型的回答: D
索引: 3
问题: 
        十月革命之前，列宁总结当时变化了的新情况，得出“社会主义可能首先在少数甚至在单独一个资本主文国家内获得胜利”的结论。列宁得出这个结论的理论依据是( )
        A: 资本主义经济和政治发展不平衡规律 
        B: 资本主义国家的不断衰落
        C: 科学社会主义理论的广泛传播 
        D: 无产阶级力量的不断壮大
        
正确答案: A
模型的回答: A
索引: 4
问题: 
        “什么是社会主义?

TypeError: Object of type QuestionResult is not JSON serializable

索引: 0
问题: 
        2019 年 4 月 10 日晚 9 点，全球多国科研人员合作的“事件视界望远镜”项目在全球六地同步举行新闻发布会，发布了世界上首张黑洞图像，公布了人类首次拍到的黑洞照片，这 是继 2015 年人类通过引力波视测“听到”两个“黑洞”合体之后，证明黑洞存在的直接“视界” 证据。有科学家认为，这张看起来有点模糊的照片意义非凡，它再次验证了爱因斯坦广义相 对论的预言是对的，并将进一步帮助科学家解答星系演化等一系列宇宙本质问题。人类首次 “看到”黑洞正面照表明( )
        A: 空间的性质依赖于物质的分布及其运动状态  
        B: 世界是物质的统一体
        C: 物质世界的客观存在与人的实践和认识水平有关  
        D: 空间的观念随着人的认识发展而拓展
        
正确答案: AB
模型的回答: AD
索引: 1
问题: 
        2019 年 10 月 24 日，中共中央政治局就区块链技术发展现状和趋势进行集体学习。习近平在主持学习时指出， 我们要提高运用和管理区块链技术能力，使区块链技术在建设网络强国、发展数字经济、助力经济社会发展等方面发挥更大作用”。有专家认为，基于区块链技术，人们可以构建在没有中介辅助下多个参与方之间的资产交易、价值传递的网络，通过建立执行智能合约，推动契约关系和规则的维护和履行，降低信用建立成本，营造良好市场环境。区块链技术所实现的人与人之间相互联系的延伸和变化，表明人们的现实社会关系( )
        A: 一种单纯的经济和商业关系 
        B: 具有多样性和复杂性的特征 
        C: 归根到底受生产关系的制约 
        D: 由信息技术构造的虚拟关系
        
正确答案: BC
模型的回答: BCD
索引: 2
问题: 
        一方水土孕育一方文化。很早以前，人们就在平原、高原、山地、河谷，海域等广阔的地理空间发展出农耕、渔猎、游牧等不同经济形态，进而形成具有明显地域差别的文化。高 铁、互联网等虽然极大消除了不同地域空间之间的界限，加速地域文化的同质性转化，但地 域文化并没有消失，还在持续对当代社会发展产生重要影响。“一方水土孕育一方文化“表明
        A: 社会经济状况决定文化发展形态  